In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau




In [5]:
!pip install patool

ERROR: Could not find a version that satisfies the requirement patool (from versions: none)
ERROR: No matching distribution found for patool


In [6]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/output.rar", outdir="/content/output")

ModuleNotFoundError: No module named 'patoolib'

In [6]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [11]:

train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2)



In [13]:
size=8

In [14]:
train_data= train_datagen.flow_from_directory(r'/content/output/output/train',
                                target_size=(80,80),batch_size=size,class_mode='categorical',subset='training' )


Found 67917 images belonging to 2 classes.


In [23]:
val_datagen = ImageDataGenerator(rescale=1./255)

validation_data = val_datagen.flow_from_directory(
        '/content/output/output/val',
        target_size=(80,80),
        batch_size=size,
        class_mode='categorical',
        
)

Found 8489 images belonging to 2 classes.


In [24]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'/content/output/output/test',
                                target_size=(80,80),batch_size=size,class_mode='categorical')

Found 8492 images belonging to 2 classes.


In [27]:
bmodel = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
model = bmodel.output
model = Flatten()(model)
model = Dense(64, activation='relu')(model)
model = Dropout(0.5)(model)
model = Dense(2,activation= 'softmax')(model)

model = Model(inputs=bmodel.input, outputs= model)
for layer in bmodel.layers:
    layer.trainable = False

In [28]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 80, 80, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 80, 80, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 40, 40, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 40, 40, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 40, 40, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 20, 20, 128)       0     

In [30]:
ckpt = ModelCheckpoint(r'/content/model/model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

estop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

lr = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[ckpt,estop,lr]

In [31]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//size,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//size,
                   callbacks=callbacks,
                    epochs=50)

<ipython-input-31-4576f270b915>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//size,


Epoch 1/50
8488/8489 [============================>.] - ETA: 0s - loss: 0.4539 - accuracy: 0.7790
Epoch 1: val_loss improved from inf to 0.29020, saving model to /content/model/model.h5
8489/8489 [==============================] - 244s 28ms/step - loss: 0.4539 - accuracy: 0.7790 - val_loss: 0.2902 - val_accuracy: 0.8651 - lr: 0.0010
Epoch 2/50
8488/8489 [============================>.] - ETA: 0s - loss: 0.3897 - accuracy: 0.8263
Epoch 2: val_loss improved from 0.29020 to 0.27349, saving model to /content/model/model.h5
8489/8489 [==============================] - 226s 27ms/step - loss: 0.3897 - accuracy: 0.8263 - val_loss: 0.2735 - val_accuracy: 0.8791 - lr: 0.0010
Epoch 3/50
8489/8489 [==============================] - ETA: 0s - loss: 0.3663 - accuracy: 0.8374
Epoch 3: val_loss improved from 0.27349 to 0.23544, saving model to /content/model/model.h5
8489/8489 [==============================] - 226s 27ms/step - loss: 0.3663 - accuracy: 0.8374 - val_loss: 0.2354 - val_accuracy: 0.8931 

In [38]:
training_loss,training_accuracy = model.evaluate_generator(train_data)
print(training_accuracy)
print(training_loss)

<ipython-input-38-de86a5e11686>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  training_loss,training_accuracy = model.evaluate_generator(train_data)


0.9123783707618713
0.22021378576755524


In [39]:
testing_loss, testing_accuracy = model.evaluate_generator(test_data)
print(testing_accuracy)
print(testing_loss)

<ipython-input-39-565182f453ea>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  testing_loss, testing_accuracy = model.evaluate_generator(test_data)


0.9406500458717346
0.15508748590946198


In [40]:
val_loss,val_accuracy = model.evaluate_generator(validation_data)
print(val_accuracy)
print(val_loss)

<ipython-input-40-d5c3c6ff411f>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  val_loss,val_accuracy = model.evaluate_generator(validation_data)


0.9432206153869629
0.1502874344587326


hello
